In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

OK



## 共通関数定義

In [3]:
params = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
        #,'num_round': 1000
}

## インスタンス生成

In [4]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

In [5]:
params_base = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'eta': 0.1
        ,'gamma': 0.0
        ,'alpha': 0.0
        ,'lambda': 1.0
        ,'min_child_weight': 6
        ,'max_depth': 1
        ,'subsample': 0.8
        ,'colsample_bytree': 0.8
        ,'random_state': 17
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
        ,'eval_metric': 'auc'
        ,'colsample_bylevel': 0.3
        #,'num_round': 100000
}


def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "min_child_weight": trial.suggest_float("min_child_weight", 0.1, 10, log=True)
            ,"max_depth": trial.suggest_int("max_depth", 1, 15, step=1, log=False)
            ,"subsample": trial.suggest_float("subsample", 0.1, 0.95,  step =0.05, log=False)
            ,"colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 0.95,  step =0.05, log=False)
            ,'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 0.95,  step =0.05, log=False)
            ,"gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            # 余裕があれば調整する
            ,"alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True)
            ,"lambda": trial.suggest_float("lambda", 1e-6, 10.0, log=True)
            
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = xgb.XGBClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## 特徴量生成

In [6]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 234)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [7]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [8]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [10]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
### 変数選択01
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1168 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1122 features are remained after removing non-informative features
123 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 999 features are remained after removing features not interesting for LightGBM classifier
---=> 58 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1057 features

---=> df final shape: (356250, 1057)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,0,0,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,0,0,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,0,0,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,1,1,0,0,0,1,0,1,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,0,0,0,0,0,0,0


In [13]:
### 変数選択02
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1057 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1051 features are remained after removing non-informative features
40 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 1011 features are remained after removing features not interesting for LightGBM classifier
---=> 19 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1030 features

---=> df final shape: (356250, 1030)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


In [14]:
### 変数選択03
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1030 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1026 features are remained after removing non-informative features
15 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 1011 features are remained after removing features not interesting for LightGBM classifier
---=> 19 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1030 features

---=> df final shape: (356250, 1030)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


In [15]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


## 学習

### 001 baseline

In [16]:
# split train, and test datasets
df = df.replace([np.inf, -np.inf], np.nan)
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1030)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,1,1,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,0,0,1,1,0,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0,0,1,1,0,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0,1,1,0,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
train_setB1a = pd.read_csv('./home-credit-default-risk/exports/train_setB1a.csv')
test_setB1a =  pd.read_csv('./home-credit-default-risk/exports/test_setB1a.csv')
test_setB1a.head()

,SK_ID_CURR,setB1a
0,100001,0.033433
1,100005,0.098996
2,100013,0.042185
3,100028,0.039429
4,100038,0.161166


In [18]:
app_train = app_train.merge(train_setB1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_setB1a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,1,1,0,0,0,0,0,0.033433
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,1,1,0,0,0,0,0,0.098996
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0,1,1,0,1,0,1,0,0.042185
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,0,1,1,0,1,0,1,0,0.039429
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,1,0,0,0,0,0,0.161166


In [19]:
train_setB1xgb = pd.read_csv('./home-credit-default-risk/exports/train_setB1xgb.csv')
test_setB1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_setB1xgb.csv')
test_setB1xgb.head()

,SK_ID_CURR,setB1x
0,100001,0.032762
1,100005,0.100620
2,100013,0.044999
3,100028,0.034211
4,100038,0.172532


In [20]:
app_train = app_train.merge(train_setB1xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_setB1xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.033433,0.032762
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,0,0,0,0.098996,0.100620
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,1,0,1,0,0.042185,0.044999
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,1,0,1,0,0.039429,0.034211
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.161166,0.172532


In [21]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [22]:
app_train = app_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,1,0,0,0,0,0,0.033433,0.032762,0.040068
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,1,0,0,0,0,0,0.098996,0.100620,0.095240
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.042185,0.044999,0.044435
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.039429,0.034211,0.036586
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,1,0,0,0,0,0,0.161166,0.172532,0.141754


In [23]:
train_set1xgb = pd.read_csv('./home-credit-default-risk/exports/train_set1xgb.csv')
test_set1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set1xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [24]:
app_train = app_train.merge(train_set1xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set1xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,0,0,0,0.033433,0.032762,0.040068,0.035234
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,0,0,0,0,0,0.098996,0.100620,0.095240,0.107426
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,1,0,1,0,0.042185,0.044999,0.044435,0.048029
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,1,0,1,0,0.039429,0.034211,0.036586,0.032975
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,1,0,0,0,0,0,0.161166,0.172532,0.141754,0.161962


In [25]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.042769
1,100005,0.103694
2,100013,0.036006
3,100028,0.048812
4,100038,0.173045


In [26]:
app_train = app_train.merge(train_set2a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0,0,0.033433,0.032762,0.040068,0.035234,0.042769
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103694
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,0.042185,0.044999,0.044435,0.048029,0.036006
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045


In [27]:
train_set2nn = pd.read_csv('./home-credit-default-risk/exports/train_set2nn.csv')
test_set2nn =  pd.read_csv('./home-credit-default-risk/exports/test_set2nn.csv')
test_set2nn.head()

,SK_ID_CURR,set2nn
0,100001,0.048972
1,100005,0.141351
2,100013,0.028088
3,100028,0.039056
4,100038,0.192637


In [28]:
app_train = app_train.merge(train_set2nn, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2nn, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637


In [29]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

,SK_ID_CURR,set3a
0,100001,0.049120
1,100005,0.115729
2,100013,0.064166
3,100028,0.038931
4,100038,0.195575


In [30]:
app_train = app_train.merge(train_set3a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575


In [31]:
train_set3xgb = pd.read_csv('./home-credit-default-risk/exports/train_set3xgb.csv')
test_set3xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set3xgb.csv')
test_set3xgb.head()

,SK_ID_CURR,set3x
0,100001,0.034310
1,100005,0.127556
2,100013,0.044872
3,100028,0.030915
4,100038,0.201295


In [32]:
app_train = app_train.merge(train_set3xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [33]:
train_set4a = pd.read_csv('./home-credit-default-risk/exports/train_set4a.csv')
test_set4a =  pd.read_csv('./home-credit-default-risk/exports/test_set4a.csv')
test_set4a.head()

,SK_ID_CURR,set4a
0,100001,0.042113
1,100005,0.132192
2,100013,0.057271
3,100028,0.042206
4,100038,0.195048


In [34]:
#app_train = app_train.merge(train_set4a, how='left', on='SK_ID_CURR')
#app_test = app_test.merge(test_set4a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [35]:
train_set4xgb = pd.read_csv('./home-credit-default-risk/exports/train_set4xgb.csv')
test_set4xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set4xgb.csv')
test_set4xgb.head()

,SK_ID_CURR,set4x
0,100001,0.034978
1,100005,0.118268
2,100013,0.039774
3,100028,0.037283
4,100038,0.164890


In [36]:
#app_train = app_train.merge(train_set4xgb, how='left', on='SK_ID_CURR')
#app_test = app_test.merge(test_set4xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295


In [37]:
train_set5a = pd.read_csv('./home-credit-default-risk/exports/train_set5a.csv')
test_set5a =  pd.read_csv('./home-credit-default-risk/exports/test_set5a.csv')
test_set4a.head()

,SK_ID_CURR,set4a
0,100001,0.042113
1,100005,0.132192
2,100013,0.057271
3,100028,0.042206
4,100038,0.195048


In [38]:
app_train = app_train.merge(train_set5a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set5a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310,0.034370
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556,0.116919
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872,0.037191
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915,0.051958
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295,0.141608


In [39]:
train_set5xgb = pd.read_csv('./home-credit-default-risk/exports/train_set5xgb.csv')
test_set5xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set5xgb.csv')
test_set5xgb.head()

,SK_ID_CURR,set5x
0,100001,0.028969
1,100005,0.116288
2,100013,0.041057
3,100028,0.045375
4,100038,0.136362


In [40]:
app_train = app_train.merge(train_set5xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set5xgb, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295,0.141608,0.136362


In [41]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310,0.034370,0.028969
1,1,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556,0.116919,0.116288
2,2,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872,0.037191,0.041057
3,3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915,0.051958,0.045375
4,4,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295,0.141608,0.136362


In [42]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295,0.141608,0.136362


In [43]:
app_train.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.283935,0.234856,0.260928,0.199398,0.242499,0.240473,0.223662,0.259405,0.196429,0.244814
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.023060,0.023065,0.012171,0.012061,0.017857,0.014736,0.024137,0.023080,0.013261,0.014884
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.032398,0.038719,0.039950,0.047440,0.033203,0.029793,0.035314,0.033290,0.030715,0.036912
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.028579,0.034008,0.030776,0.030071,0.048900,0.031804,0.031135,0.022860,0.034176,0.037270
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.023252,0.027813,0.035088,0.023555,0.043556,0.060946,0.035167,0.033131,0.038256,0.035143


In [44]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [45]:
### 学習モデル構築
train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 1038) (61502, 1038)
[23:10:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.79800	validation_1-auc:0.78971
[1]	validation_0-auc:0.80269	validation_1-auc:0.79114
[2]	validation_0-auc:0.80515	validation_1-auc:0.79108
[3]	validation_0-auc:0.80694	validation_1-auc:0.79176
[4]	validation_0-auc:0.80887	validation_1-auc:0.79232
[5]	validation_0-auc:0.81053	validation_1-auc:0.79245
[6]	validation_0-auc:0.81183	validation_1-auc:0.79298
[7]	validation_0-auc:0.81300	validation_1-auc:0.79281
[8]	validation_0-auc:0.81430	validation_1-auc:0.79285
[9]	val

[31]	validation_0-auc:0.83950	validation_1-auc:0.79414
[32]	validation_0-auc:0.84081	validation_1-auc:0.79426
[33]	validation_0-auc:0.84196	validation_1-auc:0.79415
[34]	validation_0-auc:0.84365	validation_1-auc:0.79390
[35]	validation_0-auc:0.84500	validation_1-auc:0.79381
[36]	validation_0-auc:0.84581	validation_1-auc:0.79360
[37]	validation_0-auc:0.84684	validation_1-auc:0.79345
[38]	validation_0-auc:0.84822	validation_1-auc:0.79342
[39]	validation_0-auc:0.84925	validation_1-auc:0.79342
[40]	validation_0-auc:0.85058	validation_1-auc:0.79339
[41]	validation_0-auc:0.85191	validation_1-auc:0.79334
[42]	validation_0-auc:0.85321	validation_1-auc:0.79348
[43]	validation_0-auc:0.85444	validation_1-auc:0.79333
[44]	validation_0-auc:0.85602	validation_1-auc:0.79334
[45]	validation_0-auc:0.85706	validation_1-auc:0.79334
[46]	validation_0-auc:0.85839	validation_1-auc:0.79329
[47]	validation_0-auc:0.85954	validation_1-auc:0.79324
[48]	validation_0-auc:0.86092	validation_1-auc:0.79324
[49]	valid

[71]	validation_0-auc:0.88006	validation_1-auc:0.78810
[72]	validation_0-auc:0.88099	validation_1-auc:0.78802
[73]	validation_0-auc:0.88148	validation_1-auc:0.78798
[74]	validation_0-auc:0.88253	validation_1-auc:0.78783
[75]	validation_0-auc:0.88335	validation_1-auc:0.78778
[76]	validation_0-auc:0.88374	validation_1-auc:0.78768
[77]	validation_0-auc:0.88438	validation_1-auc:0.78767
[78]	validation_0-auc:0.88476	validation_1-auc:0.78763
[79]	validation_0-auc:0.88510	validation_1-auc:0.78759
[80]	validation_0-auc:0.88553	validation_1-auc:0.78760
[81]	validation_0-auc:0.88634	validation_1-auc:0.78756
[82]	validation_0-auc:0.88739	validation_1-auc:0.78765
[83]	validation_0-auc:0.88770	validation_1-auc:0.78767
[84]	validation_0-auc:0.88860	validation_1-auc:0.78763
[85]	validation_0-auc:0.88912	validation_1-auc:0.78773
[86]	validation_0-auc:0.88942	validation_1-auc:0.78766
[87]	validation_0-auc:0.89003	validation_1-auc:0.78755
[88]	validation_0-auc:0.89113	validation_1-auc:0.78750
[89]	valid

[1]	validation_0-auc:0.80214	validation_1-auc:0.79519
[2]	validation_0-auc:0.80461	validation_1-auc:0.79633
[3]	validation_0-auc:0.80689	validation_1-auc:0.79668
[4]	validation_0-auc:0.80849	validation_1-auc:0.79674
[5]	validation_0-auc:0.80956	validation_1-auc:0.79679
[6]	validation_0-auc:0.81143	validation_1-auc:0.79680
[7]	validation_0-auc:0.81255	validation_1-auc:0.79694
[8]	validation_0-auc:0.81339	validation_1-auc:0.79704
[9]	validation_0-auc:0.81472	validation_1-auc:0.79702
[10]	validation_0-auc:0.81627	validation_1-auc:0.79701
[11]	validation_0-auc:0.81733	validation_1-auc:0.79688
[12]	validation_0-auc:0.81851	validation_1-auc:0.79684
[13]	validation_0-auc:0.81937	validation_1-auc:0.79677
[14]	validation_0-auc:0.82003	validation_1-auc:0.79673
[15]	validation_0-auc:0.82093	validation_1-auc:0.79680
[16]	validation_0-auc:0.82172	validation_1-auc:0.79692
[17]	validation_0-auc:0.82284	validation_1-auc:0.79676
[18]	validation_0-auc:0.82410	validation_1-auc:0.79684
[19]	validation_0-a

In [46]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1035,set5x,0.027401,0.004612
1034,set5a,0.024781,0.002401
1037,setB1x,0.017102,0.002530
1028,set1,0.012156,0.000859
1032,set3a,0.011264,0.001359
1030,set2a,0.011057,0.001910
1029,set1x,0.006727,0.000583
1033,set3x,0.006356,0.001259
1036,setB1a,0.003048,0.000910
1031,set2nn,0.002712,0.000122


In [47]:
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295,0.141608,0.136362


In [48]:
app_train.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.283935,0.234856,0.260928,0.199398,0.242499,0.240473,0.223662,0.259405,0.196429,0.244814
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.023060,0.023065,0.012171,0.012061,0.017857,0.014736,0.024137,0.023080,0.013261,0.014884
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.032398,0.038719,0.039950,0.047440,0.033203,0.029793,0.035314,0.033290,0.030715,0.036912
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.028579,0.034008,0.030776,0.030071,0.048900,0.031804,0.031135,0.022860,0.034176,0.037270
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.023252,0.027813,0.035088,0.023555,0.043556,0.060946,0.035167,0.033131,0.038256,0.035143


In [49]:
# データセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [50]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [51]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSourceXgb2-1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.031610
1,100005,0.111530
2,100013,0.044441
3,100028,0.035599
4,100038,0.136990


### 正規化＋対数変換

In [52]:
app_train_ord = app_train.copy()
app_test_ord = app_test.copy()
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295,0.141608,0.136362


In [53]:
# infをnanへ置換
app_train_ord = app_train_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord = app_test_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295,0.141608,0.136362


In [54]:
### 欠損値補完
#app_train_ord = app_train_ord.fillna(app_train_ord.max() + (app_train_ord.max() * 0.4))
app_train_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.283935,0.234856,0.260928,0.199398,0.242499,0.240473,0.223662,0.259405,0.196429,0.244814
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.023060,0.023065,0.012171,0.012061,0.017857,0.014736,0.024137,0.023080,0.013261,0.014884
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.032398,0.038719,0.039950,0.047440,0.033203,0.029793,0.035314,0.033290,0.030715,0.036912
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.028579,0.034008,0.030776,0.030071,0.048900,0.031804,0.031135,0.022860,0.034176,0.037270
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.023252,0.027813,0.035088,0.023555,0.043556,0.060946,0.035167,0.033131,0.038256,0.035143


In [55]:
### 欠損値補完
#app_test_ord = app_test_ord.fillna(app_test_ord.max() + (app_test_ord.max() * 0.4))
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.048972,0.049120,0.034310,0.034370,0.028969
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.141351,0.115729,0.127556,0.116919,0.116288
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.028088,0.064166,0.044872,0.037191,0.041057
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.039056,0.038931,0.030915,0.051958,0.045375
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.192637,0.195575,0.201295,0.141608,0.136362


In [56]:
# infをnanへ置換
app_train_ord = FeatureEngineering2.reduce_mem_usage(app_train_ord)
app_test_ord = FeatureEngineering2.reduce_mem_usage(app_test_ord)
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.033447,0.032776,0.040070,0.035248,0.042755,0.048981,0.049133,0.034302,0.034363,0.028976
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.098999,0.100647,0.095215,0.107422,0.103699,0.141357,0.115723,0.127563,0.116943,0.116272
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.042175,0.045013,0.044434,0.048035,0.036011,0.028091,0.064148,0.044861,0.037201,0.041046
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.039429,0.034210,0.036591,0.032990,0.048798,0.039062,0.038940,0.030914,0.051971,0.045380
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.161133,0.172485,0.141724,0.161987,0.173096,0.192627,0.195557,0.201294,0.141602,0.136353


In [57]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(app_test_ord.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [58]:
### Yao-Jhonson変換
#for column in encoding_columns:
#    modules.process_yj_convert(app_train_ord, app_test_ord, column, [column])

app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0.033447,0.032776,0.040070,0.035248,0.042755,0.048981,0.049133,0.034302,0.034363,0.028976
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0.098999,0.100647,0.095215,0.107422,0.103699,0.141357,0.115723,0.127563,0.116943,0.116272
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0.042175,0.045013,0.044434,0.048035,0.036011,0.028091,0.064148,0.044861,0.037201,0.041046
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0.039429,0.034210,0.036591,0.032990,0.048798,0.039062,0.038940,0.030914,0.051971,0.045380
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0.161133,0.172485,0.141724,0.161987,0.173096,0.192627,0.195557,0.201294,0.141602,0.136353


In [59]:
### indexを設定
app_train_ord = app_train_ord.set_index('SK_ID_CURR')
app_test_ord = app_test_ord.set_index('SK_ID_CURR')
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0.033447,0.032776,0.040070,0.035248,0.042755,0.048981,0.049133,0.034302,0.034363,0.028976
100005,1,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0.098999,0.100647,0.095215,0.107422,0.103699,0.141357,0.115723,0.127563,0.116943,0.116272
100013,2,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0.042175,0.045013,0.044434,0.048035,0.036011,0.028091,0.064148,0.044861,0.037201,0.041046
100028,3,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0.039429,0.034210,0.036591,0.032990,0.048798,0.039062,0.038940,0.030914,0.051971,0.045380
100038,4,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0.161133,0.172485,0.141724,0.161987,0.173096,0.192627,0.195557,0.201294,0.141602,0.136353


In [60]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train_ord, app_test_ord, num_cols)
        
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1.732045,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,...,-0.490723,-0.485840,-0.419189,-0.450684,-0.402832,-0.317139,-0.328125,-0.469482,-0.475830,-0.514648
100005,-1.732034,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,...,0.188843,0.226562,0.158813,0.319824,0.256592,0.678223,0.364990,0.507324,0.374756,0.403564
100013,-1.732023,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,0.247943,...,-0.400146,-0.357422,-0.373535,-0.314209,-0.475830,-0.541992,-0.171875,-0.358887,-0.446533,-0.387695
100028,-1.732011,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,2.805819,...,-0.428711,-0.470947,-0.455566,-0.474609,-0.337402,-0.423828,-0.434326,-0.504883,-0.294434,-0.342285
100038,-1.732000,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,0.235763,...,0.833496,0.980957,0.646484,0.902344,1.007812,1.230469,1.195312,1.279297,0.628418,0.614746


In [61]:
### indexを削除
app_train_ord = app_train_ord.reset_index()
app_test_ord = app_test_ord.reset_index()
app_test_ord.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,setB1a,setB1x,set1,set1x,set2a,set2nn,set3a,set3x,set5a,set5x
0,100001,-1.732045,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,...,-0.490723,-0.485840,-0.419189,-0.450684,-0.402832,-0.317139,-0.328125,-0.469482,-0.475830,-0.514648
1,100005,-1.732034,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,...,0.188843,0.226562,0.158813,0.319824,0.256592,0.678223,0.364990,0.507324,0.374756,0.403564
2,100013,-1.732023,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,...,-0.400146,-0.357422,-0.373535,-0.314209,-0.475830,-0.541992,-0.171875,-0.358887,-0.446533,-0.387695
3,100028,-1.732011,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,...,-0.428711,-0.470947,-0.455566,-0.474609,-0.337402,-0.423828,-0.434326,-0.504883,-0.294434,-0.342285
4,100038,-1.732000,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,...,0.833496,0.980957,0.646484,0.902344,1.007812,1.230469,1.195312,1.279297,0.628418,0.614746


In [62]:
x_train = app_train_ord.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train_ord['TARGET']
id_train = app_train_ord[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [63]:
# データセット作成
x_test = app_test_ord.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test_ord[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [64]:
#del app_train_ord
#del app_test_ord
#gc.collect()

In [65]:
### 学習モデル構築
train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 1038) (61502, 1038)
[23:13:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.79780	validation_1-auc:0.78972
[1]	validation_0-auc:0.80238	validation_1-auc:0.79141
[2]	validation_0-auc:0.80484	validation_1-auc:0.79146
[3]	validation_0-auc:0.80673	validation_1-auc:0.79154
[4]	validation_0-auc:0.80840	validation_1-auc:0.79214
[5]	validation_0-auc:0.80999	validation_1-auc:0.79234
[6]	validation_0-auc:0.81113	validation_1-auc:0.79281
[7]	validation_0-auc:0.81242	validation_1-auc:0.79264
[8]	validation_0-auc:0.81378	validation_1-auc:0.79256
[9]	val

[31]	validation_0-auc:0.83893	validation_1-auc:0.79478
[32]	validation_0-auc:0.84020	validation_1-auc:0.79493
[33]	validation_0-auc:0.84126	validation_1-auc:0.79494
[34]	validation_0-auc:0.84276	validation_1-auc:0.79499
[35]	validation_0-auc:0.84414	validation_1-auc:0.79503
[36]	validation_0-auc:0.84507	validation_1-auc:0.79478
[37]	validation_0-auc:0.84643	validation_1-auc:0.79471
[38]	validation_0-auc:0.84779	validation_1-auc:0.79468
[39]	validation_0-auc:0.84877	validation_1-auc:0.79470
[40]	validation_0-auc:0.84998	validation_1-auc:0.79450
[41]	validation_0-auc:0.85127	validation_1-auc:0.79429
[42]	validation_0-auc:0.85270	validation_1-auc:0.79436
[43]	validation_0-auc:0.85412	validation_1-auc:0.79433
[44]	validation_0-auc:0.85595	validation_1-auc:0.79434
[45]	validation_0-auc:0.85686	validation_1-auc:0.79429
[46]	validation_0-auc:0.85799	validation_1-auc:0.79431
[47]	validation_0-auc:0.85918	validation_1-auc:0.79440
[48]	validation_0-auc:0.86048	validation_1-auc:0.79443
[49]	valid

[71]	validation_0-auc:0.88029	validation_1-auc:0.78881
[72]	validation_0-auc:0.88080	validation_1-auc:0.78880
[73]	validation_0-auc:0.88124	validation_1-auc:0.78871
[74]	validation_0-auc:0.88192	validation_1-auc:0.78869
[75]	validation_0-auc:0.88276	validation_1-auc:0.78865
[76]	validation_0-auc:0.88349	validation_1-auc:0.78862
[77]	validation_0-auc:0.88432	validation_1-auc:0.78856
[78]	validation_0-auc:0.88509	validation_1-auc:0.78860
[79]	validation_0-auc:0.88567	validation_1-auc:0.78839
[80]	validation_0-auc:0.88646	validation_1-auc:0.78842
[81]	validation_0-auc:0.88731	validation_1-auc:0.78832
[82]	validation_0-auc:0.88772	validation_1-auc:0.78833
[83]	validation_0-auc:0.88811	validation_1-auc:0.78839
[84]	validation_0-auc:0.88883	validation_1-auc:0.78838
[85]	validation_0-auc:0.88929	validation_1-auc:0.78842
[86]	validation_0-auc:0.88980	validation_1-auc:0.78827
[87]	validation_0-auc:0.89057	validation_1-auc:0.78822
[88]	validation_0-auc:0.89107	validation_1-auc:0.78824
[89]	valid

[1]	validation_0-auc:0.80132	validation_1-auc:0.79509
[2]	validation_0-auc:0.80401	validation_1-auc:0.79605
[3]	validation_0-auc:0.80603	validation_1-auc:0.79653
[4]	validation_0-auc:0.80766	validation_1-auc:0.79655
[5]	validation_0-auc:0.80912	validation_1-auc:0.79672
[6]	validation_0-auc:0.81107	validation_1-auc:0.79651
[7]	validation_0-auc:0.81247	validation_1-auc:0.79660
[8]	validation_0-auc:0.81358	validation_1-auc:0.79682
[9]	validation_0-auc:0.81488	validation_1-auc:0.79671
[10]	validation_0-auc:0.81636	validation_1-auc:0.79669
[11]	validation_0-auc:0.81720	validation_1-auc:0.79664
[12]	validation_0-auc:0.81823	validation_1-auc:0.79644
[13]	validation_0-auc:0.81892	validation_1-auc:0.79643
[14]	validation_0-auc:0.81999	validation_1-auc:0.79632
[15]	validation_0-auc:0.82075	validation_1-auc:0.79648
[16]	validation_0-auc:0.82162	validation_1-auc:0.79639
[17]	validation_0-auc:0.82256	validation_1-auc:0.79626
[18]	validation_0-auc:0.82376	validation_1-auc:0.79626
[19]	validation_0-a

In [66]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1035,set5x,0.025072,0.005341
1034,set5a,0.024177,0.003832
1037,setB1x,0.016879,0.004803
1028,set1,0.011788,0.000835
1030,set2a,0.011716,0.002030
1032,set3a,0.011445,0.002380
1029,set1x,0.006676,0.000476
1033,set3x,0.006158,0.000789
1036,setB1a,0.002940,0.000686
1031,set2nn,0.002577,0.000131


In [67]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [68]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSourceXgb2-2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.033538
1,100005,0.114836
2,100013,0.040002
3,100028,0.043327
4,100038,0.143078


## パラメータチューニング

In [69]:
#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [70]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [71]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [72]:
#train_oof, imp, metrics = baseline.train_xgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [73]:
### 推論用データセットの作成
#x_test = app_test.drop(columns=['SK_ID_CURR'])
#id_test = app_test[['SK_ID_CURR']]

### カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

### predict
#test_pred = baseline.predict_xgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

### make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_XgbHyperParameterTuningXgb2-3.csv", index=None)

In [74]:
#del app_train_ord
#del app_test_ord
#gc.collect()